In [1]:
import json

# JSON 파일 읽기
with open("../app/data/book_embeddings_openai.json", "r", encoding="utf-8") as file:
    data = json.load(file)  # JSON 데이터를 Python 객체(dict 또는 list)로 변환


In [83]:
import numpy as np
vector = []
isbn = []

book_idx = list(data.keys())
for idx in book_idx: 
  vector.append(data[idx]['embedding'])
  isbn.append(data[idx]['isbn'])
np_vector = np.array(vector) # book x embedding size

In [121]:
def gen_user_vector(embedding_matrix):
  num_rows, num_cols = embedding_matrix.shape

  # 각 열에서 랜덤한 행을 선택
  random_row_indices = np.random.randint(0, num_rows, size=num_cols)
  selected_values = embedding_matrix[random_row_indices, np.arange(num_cols)]  # 각 열에서 랜덤한 값 선택

  # 최종 (1 × N) 벡터 생성
  final_vector = selected_values.reshape(1, num_cols)[0]
  return final_vector
user_vector1 = gen_user_vector(np_vector)
user_vector2 = gen_user_vector(np_vector)

In [ ]:
# 선택되지 않은 벡터들을 제외해 나가는 방식

In [103]:
def isbn2vec(isbn_num, isbn_list, embedding_matrix):
  return embedding_matrix[isbn_list.index(isbn_num)]
isbn2vec(9791191114591, isbn, np_vector)

In [ ]:
# 유저 벡터로 부터 2개의 방향을 정한다.

In [105]:
# 유저가 특정 책에 대해서 yes, no할 때 유저 벡터가 업데이트되고 두권을 리턴

array([-0.01067669, -0.01538632, -0.01866706, ..., -0.01404888,
       -0.00756742, -0.00204618])

In [136]:
np_vector.shape

(1781, 3072)

In [ ]:
# 임베딩 벡터의 평균
embedding_mean_vector = np_vector.mean(axis = 0)

In [160]:
alpha = 0.5
a_vector = user_vector + alpha * embedding_mean_vector
b_vector = user_vector - alpha * embedding_mean_vector

In [161]:
# 선택한 책 벡터를 바탕으로 유저 벡터를 업데이트
alpha = 0.5 # 학습률
select_true_bookv = a_vector
select_false_bookv = b_vector
user_vector = user_vector + alpha * (select_true_bookv - user_vector) - alpha*( select_false_bookv- user_vector)

In [163]:
import numpy as np

def cosine_similarity_matrix(matrix, vector):
    """(1781 × 3072) 벡터 집합과 (1 × 3072) 벡터 간 코사인 유사도 계산"""
    # 각 벡터와의 내적 계산
    dot_product = np.dot(matrix, vector.T)  # (1781, 3072) ⋅ (3072,) → (1781,)
    
    # 벡터 크기 (Norm) 계산
    norm_matrix = np.linalg.norm(matrix, axis=1)  # (1781,)
    norm_vector = np.linalg.norm(vector)  # (1,)

    # 코사인 유사도 계산
    similarity = dot_product / (norm_matrix * norm_vector)

    return similarity  # (1781,) 형태의 유사도 배열 반환

# 코사인 유사도 계산
similarity_scores = cosine_similarity_matrix(np_vector, b_vector)

# 가장 유사한 벡터 찾기
most_similar_index = np.argmax(similarity_scores)  # 유사도 최대값의 인덱스
print(f"가장 유사한 벡터 인덱스: {most_similar_index}")
print(f"가장 높은 코사인 유사도: {similarity_scores[most_similar_index]}")

# 만약 같으면 두번째 유사한걸로 수정
# 이전에 사용한 질문이면 제외


가장 유사한 벡터 인덱스: 542
가장 높은 코사인 유사도: 0.50581282956619
